## Carregando dados de um arquivo para um Dataframe

In [1]:
import findspark

findspark.init()

from pyspark.sql import SparkSession

# Cria uma instância de SparkSession, que é a entrada para usar o Spark
spark = SparkSession.builder \
    .master('local[*]') \
    .appName("Iniciando com Spark") \
    .getOrCreate()
    
spark

In [2]:
#criando um objeto sparksession object e um appName 
spark=SparkSession.builder.appName("fooddf").getOrCreate()

In [3]:
# Fazendo a leitura do arquivo food_coded.csv
df = spark.read.option("header", "true").csv(r"C:\Users\Kaue\Documents\Cursos\Databricks e PySpark\PySpark\data\food_coded.csv")

In [4]:
# Exibindo as 10 primeiras linhas
df.show(10)

+---------------+------+--------------------+----------------+------------+--------------+------+--------------------+--------------------+---------------------------+----+----------------------------+-------+--------------------+------------------+-----+--------------------+--------------------+---------------------+----------+----------+-----------+--------+----------------+-----------------+--------------------+-----------------+--------+--------------------+-----+---------+-----------+----------+---------------+--------------------+--------------------+----------------+------+-----------+------------+--------------+--------------+--------------------+----------------+--------------------+-----------------+-------------+------------+------------+------------+----------------------+----+------+---------+-----------------+---------------+-----------+-----------+--------+---------------+--------------------+
|            GPA|Gender|           breakfast|calories_chicken|calories_day|cal

In [5]:
# exibindo o Schema
df.printSchema()

root
 |-- GPA: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- breakfast: string (nullable = true)
 |-- calories_chicken: string (nullable = true)
 |-- calories_day: string (nullable = true)
 |-- calories_scone: string (nullable = true)
 |-- coffee: string (nullable = true)
 |-- comfort_food: string (nullable = true)
 |-- comfort_food_reasons: string (nullable = true)
 |-- comfort_food_reasons_coded9: string (nullable = true)
 |-- cook: string (nullable = true)
 |-- comfort_food_reasons_coded11: string (nullable = true)
 |-- cuisine: string (nullable = true)
 |-- diet_current: string (nullable = true)
 |-- diet_current_coded: string (nullable = true)
 |-- drink: string (nullable = true)
 |-- eating_changes: string (nullable = true)
 |-- eating_changes_coded: string (nullable = true)
 |-- eating_changes_coded1: string (nullable = true)
 |-- eating_out: string (nullable = true)
 |-- employment: string (nullable = true)
 |-- ethnic_food: string (nullable = true)
 |-- e

In [6]:
# Selecionando alguns campos
df.select('mother_profession','type_sports', 'Gender', 'weight','calories_day').show(30)

+--------------------+--------------------+------+--------------------+------------+
|   mother_profession|         type_sports|Gender|              weight|calories_day|
+--------------------+--------------------+------+--------------------+------------+
|          unemployed|          car racing|     2|                 187|         nan|
|           Nurse RN |         Basketball |     1|                 155|           3|
|       owns business|                none|     1|I'm not answering...|           4|
|                NULL|                NULL|     1|                NULL|           3|
|                NULL|                NULL|  NULL|                NULL|        NULL|
|                NULL|                NULL|     4|                NULL|           1|
|Substance Abuse C...|            Softball|     1|                 190|           2|
|        Hair Braider|               None.|     1|                 190|           3|
|          Journalist|              soccer|     2|               

In [7]:
# utilizando funcoes do pyspark para filtro
from pyspark.sql.functions import *

# Filtrando os dados que nao sao nulos
resultdf1 = df.filter(df.weight.isNotNull()).select('mother_profession','type_sports', 'Gender', 'weight','calories_day')

In [8]:
# Exibindo os dados
resultdf1.show()

+--------------------+--------------------+------+--------------------+------------+
|   mother_profession|         type_sports|Gender|              weight|calories_day|
+--------------------+--------------------+------+--------------------+------------+
|          unemployed|          car racing|     2|                 187|         nan|
|           Nurse RN |         Basketball |     1|                 155|           3|
|       owns business|                none|     1|I'm not answering...|           4|
|Substance Abuse C...|            Softball|     1|                 190|           2|
|        Hair Braider|               None.|     1|                 190|           3|
|          Journalist|              soccer|     2|                 180|           3|
|                cook|                none|     1|                 137|           3|
|Elementary School...|                none|     1|                 180|         nan|
|  Pharmaceutical rep|        field hockey|     1|               

In [9]:
# Retirando por meio de REGEX todos os carcteres que nao sejam numericos do campo weight
resultdf2 = resultdf1.filter(col("weight").rlike("^[0-9]*$")) # regex para tudo que é diferente de número
resultdf2.show()

+--------------------+--------------------+------+------+------------+
|   mother_profession|         type_sports|Gender|weight|calories_day|
+--------------------+--------------------+------+------+------------+
|          unemployed|          car racing|     2|   187|         nan|
|           Nurse RN |         Basketball |     1|   155|           3|
|Substance Abuse C...|            Softball|     1|   190|           2|
|        Hair Braider|               None.|     1|   190|           3|
|          Journalist|              soccer|     2|   180|           3|
|                cook|                none|     1|   137|           3|
|Elementary School...|                none|     1|   180|         nan|
|  Pharmaceutical rep|        field hockey|     1|   125|           3|
|     Chidos Cleaners|              soccer|     1|   116|           3|
|     Court Reporter |             Running|     1|   110|           4|
|Child care provider |Soccer and basket...|     2|   264|           3|
|     

In [10]:
# Gerando uma agregação dos dados por Gender e weight
resultdf2.groupBy("Gender", "weight").count().orderBy("weight").show()

+------+------+-----+
|Gender|weight|count|
+------+------+-----+
|     1|     1|    1|
|     1|   100|    1|
|     1|   105|    1|
|     1|   110|    1|
|     1|   112|    1|
|     1|   113|    1|
|     1|   115|    1|
|     1|   116|    1|
|     1|   120|    3|
|     1|   123|    1|
|     1|   125|    4|
|     1|   127|    1|
|     1|   128|    2|
|     1|   129|    2|
|     1|   130|    3|
|     1|   135|    7|
|     2|   135|    1|
|     1|   137|    1|
|     2|   140|    2|
|     1|   140|    6|
+------+------+-----+
only showing top 20 rows



In [13]:
# Seelcionando os registros cujo weight >= 150, indicando sobrepeso, separando por tipo de esporte
resultdf3 = resultdf2.select("type_sports", when(resultdf2.weight >= 150, "Sobrepeso").alias("weight_category"))

In [14]:
# Exibindo os dados
resultdf3.show()


+--------------------+---------------+
|         type_sports|weight_category|
+--------------------+---------------+
|          car racing|      Sobrepeso|
|         Basketball |      Sobrepeso|
|            Softball|      Sobrepeso|
|               None.|      Sobrepeso|
|              soccer|      Sobrepeso|
|                none|           NULL|
|                none|      Sobrepeso|
|        field hockey|           NULL|
|              soccer|           NULL|
|             Running|           NULL|
|Soccer and basket...|      Sobrepeso|
|intramural volley...|           NULL|
|              Hockey|      Sobrepeso|
|              Hockey|      Sobrepeso|
|                 nan|           NULL|
|              hockey|      Sobrepeso|
|            dancing |           NULL|
|          basketball|      Sobrepeso|
|              Soccer|      Sobrepeso|
|              Tennis|      Sobrepeso|
+--------------------+---------------+
only showing top 20 rows



In [15]:
# Retirando as informacoes nulas do segundo campo
resultdf4 = resultdf3.filter(resultdf3[1].isNotNull())

In [16]:
# Exibindo os dados
resultdf4.show()


+--------------------+---------------+
|         type_sports|weight_category|
+--------------------+---------------+
|          car racing|      Sobrepeso|
|         Basketball |      Sobrepeso|
|            Softball|      Sobrepeso|
|               None.|      Sobrepeso|
|              soccer|      Sobrepeso|
|                none|      Sobrepeso|
|Soccer and basket...|      Sobrepeso|
|              Hockey|      Sobrepeso|
|              Hockey|      Sobrepeso|
|              hockey|      Sobrepeso|
|          basketball|      Sobrepeso|
|              Soccer|      Sobrepeso|
|              Tennis|      Sobrepeso|
|   tennis soccer gym|      Sobrepeso|
|     Gaelic Football|      Sobrepeso|
|              Hockey|      Sobrepeso|
|           Lacrosse |      Sobrepeso|
|                 nan|      Sobrepeso|
|                 nan|      Sobrepeso|
|      none organized|      Sobrepeso|
+--------------------+---------------+
only showing top 20 rows



In [17]:
# Retirando as informacoes none, None., nan do primeiro campo ( type_sports)
resultdf4.filter((resultdf3[0] != 'none') & (resultdf3[0] != 'None.') & (resultdf3[0] != 'nan')).show()

+--------------------+---------------+
|         type_sports|weight_category|
+--------------------+---------------+
|          car racing|      Sobrepeso|
|         Basketball |      Sobrepeso|
|            Softball|      Sobrepeso|
|              soccer|      Sobrepeso|
|Soccer and basket...|      Sobrepeso|
|              Hockey|      Sobrepeso|
|              Hockey|      Sobrepeso|
|              hockey|      Sobrepeso|
|          basketball|      Sobrepeso|
|              Soccer|      Sobrepeso|
|              Tennis|      Sobrepeso|
|   tennis soccer gym|      Sobrepeso|
|     Gaelic Football|      Sobrepeso|
|              Hockey|      Sobrepeso|
|           Lacrosse |      Sobrepeso|
|      none organized|      Sobrepeso|
|            softball|      Sobrepeso|
|            Lacrosse|      Sobrepeso|
|           Softball |      Sobrepeso|
|             Dancing|      Sobrepeso|
+--------------------+---------------+
only showing top 20 rows

